In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve, f1_score
from xgboost import XGBClassifier

In [ ]:
# Configure
OUTPUT_DIR = "analysis"
DATA_FILE = "bank-additional-full.csv"
RESULTS_FILE = os.path.join(OUTPUT_DIR, "modeling_results.md")

In [ ]:
def load_data(filepath):
    return pd.read_csv(filepath, sep=';')

In [ ]:
def preprocess_data(df):
    # 1. Drop 'duration' (Leakage)
    if 'duration' in df.columns:
        print("Dropping 'duration' column (Leakage prevention)...")
        df = df.drop(columns=['duration'])
    
    # 2. Feature Engineering
    print("Performing Feature Engineering...")
    
    # 'pdays': Create binary 'was_contacted'
    # 999 means client was not previously contacted
    if 'pdays' in df.columns:
        df['was_contacted'] = (df['pdays'] < 999).astype(int)
        df = df.drop(columns=['pdays'])
    
    # 'age': Create 'is_retired' (> 60)
    if 'age' in df.columns:
        df['is_retired'] = (df['age'] > 60).astype(int)
    
    # 3. Encode Target
    le = LabelEncoder()
    df['y'] = le.fit_transform(df['y']) # yes=1, no=0
    
    # 4. Identify Features
    target = 'y'
    X = df.drop(columns=[target])
    y = df[target]
    
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    
    print(f"Categorical columns: {categorical_cols}")
    print(f"Numerical columns: {numerical_cols}")
    
    # 5. Create Preprocessing Pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
        ])
    
    return X, y, preprocessor

In [ ]:
def evaluate_model(model, X_test, y_test, model_name, f):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    print(f"Evaluating {model_name}...")
    
    f.write(f"## {model_name} Results\n\n")
    
    # Classification Report
    report = classification_report(y_test, y_pred)
    f.write("### Classification Report\n")
    f.write(f"```\n{report}\n```\n\n")
    
    # Explicit F1-Score for Class 1
    f1_class1 = f1_score(y_test, y_pred, pos_label=1)
    f.write(f"- **F1-Score (Class 1):** {f1_class1:.4f}\n\n")
    
    # ROC-AUC
    auc = roc_auc_score(y_test, y_prob)
    f.write(f"- **ROC AUC Score:** {auc:.4f}\n\n")
    
    # Precision-Recall Curve (NEW)
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, marker='.', label=model_name)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - {model_name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(OUTPUT_DIR, f"pr_curve_{model_name.replace(' ', '_')}.png"))
    plt.close()
    f.write(f"![Precision-Recall Curve {model_name}](pr_curve_{model_name.replace(' ', '_')}.png)\n\n")
    
    # Confusion Matrix Plot
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(OUTPUT_DIR, f"cm_{model_name.replace(' ', '_')}.png"))
    plt.close()
    f.write(f"![Confusion Matrix {model_name}](cm_{model_name.replace(' ', '_')}.png)\n\n")

    # Feature Importance (if available)
    if hasattr(model, 'named_steps'):
        classifier = model.named_steps['classifier']
        # Handle XGBoost feature importances
        if hasattr(classifier, 'feature_importances_'):
            preprocessor = model.named_steps['preprocessor']
            try:
                cat_names = preprocessor.named_transformers_['cat'].get_feature_names_out()
                num_names = preprocessor.named_transformers_['num'].feature_names_in_
                feature_names = np.concatenate([num_names, cat_names])
                
                importances = classifier.feature_importances_
                # Ensure lengths match (sometimes OneHot drops columns)
                if len(importances) == len(feature_names):
                    indices = np.argsort(importances)[::-1][:20] # Top 20
                    
                    plt.figure(figsize=(10, 8))
                    plt.title(f"Feature Importances - {model_name} (Top 20)")
                    plt.barh(range(len(indices)), importances[indices], align='center')
                    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
                    plt.gca().invert_yaxis()
                    plt.xlabel('Relative Importance')
                    plt.tight_layout()
                    plt.savefig(os.path.join(OUTPUT_DIR, f"fi_{model_name.replace(' ', '_')}.png"))
                    plt.close()
                    f.write(f"![Feature Importance {model_name}](fi_{model_name.replace(' ', '_')}.png)\n\n")
            except Exception as e:
                print(f"Could not plot feature importance for {model_name}: {e}")

In [ ]:
def main():
    if not os.path.exists(DATA_FILE):
        print(f"Error: {DATA_FILE} not found.")
        return

    df = load_data(DATA_FILE)
    X, y, preprocessor = preprocess_data(df)
    
    # Split Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Calculate scale_pos_weight for XGBoost
    # sum(negative) / sum(positive)
    pos_count = y_train.sum()
    neg_count = len(y_train) - pos_count
    scale_pos_weight = neg_count / pos_count
    print(f"XGBoost scale_pos_weight: {scale_pos_weight:.2f}")

    with open(RESULTS_FILE, "w") as f:
        f.write("# Modeling Results\n\n")
        
        # 1. Logistic Regression (Baseline)
        lr_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42))
        ])
        
        lr_pipeline.fit(X_train, y_train)
        evaluate_model(lr_pipeline, X_test, y_test, "Logistic Regression", f)
        
        # 2. Random Forest
        rf_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1))
        ])
        
        rf_pipeline.fit(X_train, y_train)
        evaluate_model(rf_pipeline, X_test, y_test, "Random Forest", f)
        
        # 3. XGBoost
        xgb_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', XGBClassifier(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=6,
                scale_pos_weight=scale_pos_weight,
                random_state=42,
                n_jobs=-1,
                eval_metric='logloss'
            ))
        ])
        
        xgb_pipeline.fit(X_train, y_train)
        evaluate_model(xgb_pipeline, X_test, y_test, "XGBoost", f)

    print(f"Results saved to {RESULTS_FILE}")

In [ ]:
if __name__ == "__main__":
    main()